# Weight sparsity control (Tensorflow)

This code is for weight sparsity control in MLP.  
It is written for Python 3.5/3.6 and Tensorflow 1.1.0.  
We can easily implement weight sparsity control follwing several steps described below.  

1. Import
Importing necessary modules, functions
Functiona definition
Hoyer sparseness, ReLU, RmsProp, adam
Class definition
Hidden layer, multiple layer perceptron
Parameters of dnnwsp
Epoches, learning rates ...
Input data
Train and test data set from sample data set
Build Model
Building training dnnwsp model with L1 and L2 regulariation term using trainig data and test model using test data
Learning
Training dnnwsp model
Save variables
Save important variables
logistic_sgd.py
Logistic regression is a probabilistic, linear classifier
Main function
Execution the dnnwsp code!

## 00. Import 

First of all, call modules which containing Python definitions and statements.

In [3]:
# This import statement gives Python access to all of TensorFlow's classes, methods, and symbols. 
import tensorflow as tf
# NumPy is the fundamental package for scientific computing with Python.
import numpy as np
# Linear algebra module for calculating L1 and L2 norm  
from numpy import linalg as LA
# To plot the results
import matplotlib.pyplot as plt
# To check the directory when saving the results
import os.path
# The module for file input and output
import scipy.io as sio

## 01. Parameters

You can specify either using weight sparsity control mode in layer wise or node wise.
Also, may select optimzer algorithm among five selections.
Then set the number of nodes the value of first element is for input layer, the last one for output layer,  and the others in the minddle for hidden layers. 
We can adjust learning parameters total epoch, mini-batch size, when to begin learning rate annealing, decaying rate of learning rate, initial value of learning rate, and minimum value of learning rate. Besides, learning rate of beta of weight sparsity control(for L1 regularization) and L2 parameter(for L2 regularization)

In [6]:
from customizationGUI \
        import mode, optimizer_algorithm, nodes, total_epoch, batch_size,\
        beginAnneal, decay_rate, lr_init, min_lr,lr_beta, L2_param, max_beta, tg_hsp


## 02. Input data

Dataset we provides consists of train, test, and validation parts. This part splits dataset and save them. 
Input dimension is 74484, and the number of output nodes for classification is four (Left-hand clecnhing (LH), right-hand clecnhing, auditory attention (AD), and visual stimulus (VS) tasks).

In [7]:
################ lhrhadvs_sample_data.mat ##################
# train_x  = 240 volumes x 74484 voxels  
# train_x  = 240 volumes x 1 [0:left-hand clenching task, 1:right-hand clenching task, 2:auditory task, 3:visual task]
# test_x  = 120 volumes x 74484 voxels
# test_y  = 120 volumes x 1 [0:left-hand clenching task, 1:right-hand clenching task, 2:auditory task, 3:visual task]
############################################################

datasets = sio.loadmat('lhrhadvs_sample_data.mat')


train_x = datasets['train_x']
train_y = np.zeros((np.shape(datasets['train_y'])[0],np.max(datasets['train_y'])+1))
# transform into One-hot
for i in np.arange(np.shape(datasets['train_y'])[0]):
    train_y[i][datasets['train_y'][i][0]]=1 


test_x = datasets['test_x']
test_y = np.zeros((np.shape(datasets['test_y'])[0],np.max(datasets['test_y'])+1))
# transform into One-hot
for i in np.arange(np.shape(datasets['test_y'])[0]):
    test_y[i][datasets['test_y'][i][0]]=1 


## 03. Build Model

This part builds the MLP model by concatenating all layers based on the information we provided earlier.

In [ ]:
# 'node_index' to split placeholder, for an example, given hidden_nodes=[100, 100, 100], nodes_index=[0, 100, 200, 300]
nodes_index= [int(np.sum(nodes[1:i+1])) for i in np.arange(np.shape(nodes)[0]-1)]

# Make two placeholders to fill the values later when training or testing
X=tf.placeholder(tf.float32,[None,nodes[0]])
Y=tf.placeholder(tf.float32,[None,nodes[-1]])

# Create randomly initialized weight variables 
w_init=[tf.div(tf.random_normal([nodes[i],nodes[i+1]]), tf.sqrt(float(nodes[i])/2)) for i in np.arange(np.shape(nodes)[0]-1)]
w=[tf.Variable(w_init[i], dtype=tf.float32) for i in np.arange(np.shape(nodes)[0]-1)]
# Create randomly initialized bias variables 
b=[tf.Variable(tf.random_normal([nodes[i+1]]), dtype=tf.float32) for i in np.arange(np.shape(nodes)[0]-1)]

# Build MLP model 
hidden_layers=[0.0]*(np.shape(nodes)[0]-2)
for i in np.arange(np.shape(nodes)[0]-2):
    # Input layer
    if i==0:
        hidden_layers[i]=tf.add(tf.matmul(X,w[i]),b[i])
        hidden_layers[i]=tf.nn.tanh(hidden_layers[i])
    # The other layers    
    else:     
        hidden_layers[i]=tf.add(tf.matmul(hidden_layers[i-1],w[i]),b[i])
        hidden_layers[i]=tf.nn.tanh(hidden_layers[i])
# Output layer
output_layer=tf.add(tf.matmul(hidden_layers[-1],w[-1]),b[-1])

# Logistic regression layer
logRegression_layer=tf.nn.tanh(output_layer)
                    


## 04. Function Definition

Definitions of functions to create
- beta
- L1 loss
- L2 loss
- cost
- optimzer
- sparsity control

In [ ]:

if mode=='layer':
    # Weight sparsity control with Hoyer's sparsness (Layer wise)  
    def Hoyers_sparsity_control(w_,b,max_b,tg):
        
        # Get value of weight
        W=sess.run(w_)
        [nodes,dim]=W.shape  
        
        # vectorize weight matrix 
        Wvec=W.flatten()     
        sqrt_nsamps=np.sqrt(Wvec.shape[0])
        
        # Calculate L1 and L2 norm     
        L1=LA.norm(Wvec,1)
        L2=LA.norm(Wvec,2)
        
        # Calculate hoyer's sparsness
        h=(sqrt_nsamps-(L1/L2))/(sqrt_nsamps-1)
        
        # Update beta
        b-=lr_beta*np.sign(h-tg)
        
        # Trim value
        b=0.0 if b<0.0 else b
        b=max_b if b>max_b else b
                         
        return [h,b]
    
    
elif mode=='node':   
    # Weight sparsity control with Hoyer's sparsness (Node wise)
    def Hoyers_sparsity_control(w_,b_vec,max_b,tg):
    
        # Get value of weight
        W=sess.run(w_)
        [nodes,dim]=W.shape
        sqrt_nsamps=np.sqrt(nodes)
        
        # Calculate L1 and L2 norm 
        L1=LA.norm(W,1,axis=0)
        L2=LA.norm(W,2,axis=0)
        
        h_vec = np.zeros((1,dim))
        tg_vec = np.ones(dim)*tg
        
        # Calculate hoyer's sparsness
        h_vec=(sqrt_nsamps-(L1/L2))/(sqrt_nsamps-1)
        
        # Update beta
        b_vec-=lr_beta*np.sign(h_vec-tg_vec)
        
        # Trim value
        b_vec[b_vec<0.0]=0.0
        b_vec[b_vec>max_b]=max_b
        
               
        return [h_vec,b_vec]
    


# Make placeholders for total beta array (make a long one to concatenate every beta vector) 
def init_beta():
    if mode=='layer':
        # The size is same with the number of layers
        Beta=tf.placeholder(tf.float32,[np.shape(nodes)[0]-2])
    elif mode=='node':
        # The size is same with the number of nodes
        Beta=tf.placeholder(tf.float32,[np.sum(nodes[1:-1])])

    return Beta


# Make L1 loss term for regularization
def init_L1loss():
    if mode=='layer':
        # Get L1 loss term by simply multiplying beta(scalar value) and L1 norm of weight for each layer
        L1_loss=[Beta[i]*tf.reduce_sum(abs(w[i])) for i in np.arange(np.shape(nodes)[0]-2)]
    elif mode=='node':
        # Get L1 loss term by multiplying beta(vector values as many as nodes) and L1 norm of weight for each layer
        L1_loss=[tf.reduce_mean(tf.matmul(abs(w[i]),tf.cast(tf.diag(Beta[nodes_index[i]:nodes_index[i+1]]),tf.float32))) for i in np.arange(np.shape(nodes)[0]-2)]

    return L1_loss


# Make L2 loss term for regularization
def init_L2loss():
    L2_loss=[tf.reduce_sum(tf.square(w[i])) for i in np.arange(np.shape(nodes)[0]-1)] 
    return L2_loss


       

# Define cost term (Cost = cross entropy + L1 term + L2 term )    
def init_cost():

    # A softmax regression : it adds up the evidence of our input being in certain classes, and converts that evidence into probabilities.
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logRegression_layer, labels=Y)) \
                                     + tf.reduce_sum(L1_loss) + L2_reg*tf.reduce_sum(L2_loss)   
    
    return cost


# TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function.
def init_optimizer(Lr):
    if optimizer_algorithm=='GradientDescent':
        optimizer=tf.train.GradientDescentOptimizer(Lr).minimize(cost) 
    elif optimizer_algorithm=='Adagrad':
        optimizer=tf.train.AdagradOptimizer(Lr).minimize(cost) 
    elif optimizer_algorithm=='Adam':
        optimizer=tf.train.AdamOptimizer(Lr).minimize(cost) 
    elif optimizer_algorithm=='Momentum':
        optimizer=tf.train.MomentumOptimizer(Lr).minimize(cost) 
    elif optimizer_algorithm=='RMSProp':
        optimizer=tf.train.RMSPropOptimizer(Lr).minimize(cost) 

    return optimizer



# initialization   
def init_otherVars():           
    if mode=='layer': 
        beta_val = np.zeros(np.shape(nodes)[0]-2)
        beta = np.zeros(np.shape(nodes)[0]-2)
        hsp_val = np.zeros(np.shape(nodes)[0]-2)            
        plot_beta = np.zeros(np.shape(nodes)[0]-2)
        plot_hsp = np.zeros(np.shape(nodes)[0]-2)
                   
    elif mode=='node':                       
        beta_val = [np.zeros(nodes[i+1]) for i in np.arange(np.shape(nodes)[0]-2)]  
        beta = np.zeros(np.sum(nodes[1:-1]))
        hsp_val = [np.zeros(nodes[i+1]) for i in np.arange(np.shape(nodes)[0]-2)]            
        plot_beta = [np.zeros(nodes[i+1]) for i in np.arange(np.shape(nodes)[0]-2)]
        plot_hsp = [np.zeros(nodes[i+1]) for i in np.arange(np.shape(nodes)[0]-2)]
    
    # make arrays to store and plot results
    plot_lr=np.zeros(1)
    plot_cost=np.zeros(1)
    plot_train_err=np.zeros(1)
    plot_test_err=np.zeros(1)
    
    # initialize learning rate
    lr = lr_init 
    
    
    return lr, beta_val, beta, hsp_val, plot_beta, plot_hsp, plot_lr, plot_cost, plot_train_err, plot_test_err
        





Then create them by using those functions above.

In [ ]:

# Make a placeholder for learning rate to be able to update learning rate (Learning rate decaying) 
Lr=tf.placeholder(tf.float32)


Beta = init_beta()
L1_loss = init_L1loss()
L2_loss = init_L2loss()
cost = init_cost()

optimizer=init_optimizer(Lr)

correct_prediction=tf.equal(tf.argmax(logRegression_layer,1),tf.argmax(Y,1))  
# calculate an average error depending on how frequent it classified correctly   
error=1-tf.reduce_mean(tf.cast(correct_prediction,tf.float32))      


lr, beta_val, beta, hsp_val, plot_beta, plot_hsp, plot_lr, plot_cost, plot_train_err, plot_test_err = init_otherVars()


## 05. Condition check

Check the conditions to catch some possible errors.

In [ ]:
condition=False

print()

if np.size(nodes) <3:
    print("Error : The number of total layers is not enough.")
elif (np.size(nodes)-2) != np.size(max_beta):
    print("Error : The number of hidden layers and max beta values don't match. ")
elif (np.size(nodes)-2) != np.size(tg_hsp):
    print("Error : The number of hidden layers and target sparsity values don't match.")
elif np.size(train_x,axis=0) != np.size(train_y,axis=0):
    print("Error : The sizes of input train datasets and output train datasets don't match. ")  
elif np.size(test_x,axis=0) != np.size(test_y,axis=0):
    print("Error : The sizes of input test datasets and output test datasets don't match. ")     
elif (np.any(np.array(tg_hsp)<0)) | (np.any(np.array(tg_hsp)>1)):  
    print("Error : The values of target sparsities are inappropriate.")
else:
    condition=True


## 06. Learning

If condition is satisfied, may start session where training and tesing proceed.
Firstly, initialize variables with initialization function.
Then start learning , getting cost and optimizing, for all epochs. In every epoch, training data is split into mini batches so that every learning iteration is mini batch learning.
At the end of every epoch, get training error and test error. Also, save cost, learning rate, beta, hsp and so on in order to plot them later. 

※ No need for 'feed_dict=' on Tensorflow version 1.1.0

In [ ]:
if condition==True:

    # variables are not initialized when you call tf.Variable
    # To initialize all the variables in a TensorFlow program, you must explicitly call a special operation         
    init = tf.global_variables_initializer()              
   
    

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:           
    
        # run tensorflow variable initialization
        sess.run(init)
        

        # Start training 
        for epoch in np.arange(total_epoch):            
                   
            # Shuffle training data when starting each epoch           
            total_sample = np.size(train_x, axis=0)
            sample_ids = np.arange(total_sample)
            np.random.shuffle(sample_ids) 
            
            train_x_shuff = np.array([ train_x[i] for i in sample_ids])
            train_y_shuff = np.array([ train_y[i] for i in sample_ids])
            
            
            # Begin Annealing
            if beginAnneal == 0:
                lr = lr * 1.0
            elif epoch+1 > beginAnneal:
                lr = max( min_lr, (-decay_rate*(epoch+1) + (1+decay_rate*beginAnneal)) * lr )  
            
        
            
            
            # Calculate how many mini-batch iterations we need
            total_batch = int(np.shape(train_x)[0]/batch_size) 
            
            cost_epoch=0.0
            
            # minibatch based training  
            for batch in np.arange(total_batch):
                batch_x = train_x_shuff[sample_ids[batch*batch_size:(batch+1)*batch_size]]
                batch_y = train_y_shuff[sample_ids[batch*batch_size:(batch+1)*batch_size]]
                
                # Get cost and optimize the model
                cost_batch,_=sess.run([cost,optimizer],{Lr:lr, X:batch_x, Y:batch_y, Beta:beta})

                cost_epoch+=cost_batch/total_batch      
        
        
                # weight sparsity control    
                if mode=='layer':                   
                    for i in np.arange(np.shape(nodes)[0]-2):
                        [hsp_val[i], beta_val[i]] = Hoyers_sparsity_control(w[i], beta_val[i], max_beta[i], tg_hsp[i])   
                    beta=beta_val                      

                elif mode=='node':                             
                    for i in np.arange(np.shape(nodes)[0]-2):
                        [hsp_val[i], beta_val[i]] = Hoyers_sparsity_control(w[i], beta_val[i], max_beta[i], tg_hsp[i])   
                    # flatten beta_val (shape (3, 100) -> (300,))
                    beta=[item for sublist in beta_val for item in sublist]
               
            # get train error
            train_err_epoch=sess.run(error,{X:train_x_shuff, Y:train_y_shuff})
            plot_train_err=np.hstack([plot_train_err,[train_err_epoch]])
            
            # get test error
            test_err_epoch=sess.run(error,{X:test_x, Y:test_y})
            plot_test_err=np.hstack([plot_test_err,[test_err_epoch]])
            
            
            
            # Save the results to plot at the end
            plot_lr=np.hstack([plot_lr,[lr]])
            plot_cost=np.hstack([plot_cost,[cost_epoch]])
            
            if mode=='layer':
                plot_hsp=[np.vstack([plot_hsp[i],[hsp_val[i]]]) for i in np.arange(np.shape(nodes)[0]-2)]
                plot_beta=[np.vstack([plot_beta[i],[beta[i]]]) for i in np.arange(np.shape(nodes)[0]-2)]
                
            elif mode=='node':
                plot_hsp=[np.vstack([plot_hsp[i],[np.transpose(hsp_val[i])]]) for i in np.arange(np.shape(nodes)[0]-2)]
                plot_beta=[np.vstack([plot_beta[i],[np.transpose(beta_val[i])]]) for i in np.arange(np.shape(nodes)[0]-2)]

            
            # Print cost and errors after every training epoch       
            print("< Epoch", "{:02d}".format(epoch+1),"> Cost :", "{:.3f}".format(cost_epoch)\
                                            ,"/ Train err :", "{:.3f}".format(train_err_epoch),"/ Test err :","{:.3f}".format(test_err_epoch))


        # Print final accuracy on test set
        print ("")
        print("* Test accuracy :", "{:.3f}".format(1-sess.run(error,{X:test_x, Y:test_y})))
            
else:
    # Don't run the session but print 'failed' if any condition is not met
    print("Failed!")  
     

## 07. Plot & save results

Finally, show the the results and save them as .mat file.

In [ ]:

if condition==True:
       
    # Plot the change of learning rate
    plt.title("Learning rate plot",fontsize=16)
    plot_lr=plot_lr[1:]
    plt.ylim(0.0, lr_init*1.2)
    plt.plot(plot_lr)
    plt.show()
    
    # Plot the change of cost
    plt.title("Cost plot",fontsize=16)
    plot_cost=plot_cost[1:]
    plt.plot(plot_cost)
    plt.show()   
    
 
  
    # Plot train & test error
    plt.title("Training & Test error",fontsize=16)
    plot_train_err=plot_train_err[1:]
    plt.plot(plot_train_err)
    plt.hold
    plot_test_err=plot_test_err[1:]
    plt.plot(plot_test_err)
    plt.ylim(0.0, 1.0)
    plt.legend(['Training error', 'Test error'],loc='upper right')
    plt.show() 


 
    # Plot the change of beta value
    print("")       
    for i in np.arange(np.shape(nodes)[0]-2):
        print("")
        print("                  < Hidden layer",i+1,">")
        plt.title("Beta plot",fontsize=16)
        plot_beta[i]=plot_beta[i][1:]
        plt.plot(plot_beta[i])
        plt.ylim(0.0, np.max(max_beta)*1.2)
        plt.show()
    
    
    # Plot the change of Hoyer's sparsity
    print("")            
    for i in np.arange(np.shape(nodes)[0]-2):
        print("")
        print("                  < Hidden layer",i+1,">")
        plt.title("Hoyer's sparsity plot",fontsize=16)
        plot_hsp[i]=plot_hsp[i][1:]
        plt.plot(plot_hsp[i])
        plt.ylim(0.0, 1.0)
        plt.show()
    
    # make a new 'results' directory in the current directory
    current_directory = os.getcwd()
    final_directory = os.path.join(current_directory, r'results')
    if not os.path.exists(final_directory):
        os.makedirs(final_directory) 
        
    # save results as .mat file
    sio.savemat("results/result_learningrate.mat", mdict={'lr': plot_lr})
    sio.savemat("results/result_cost.mat", mdict={'cost': plot_cost})
    sio.savemat("results/result_train_err.mat", mdict={'trainErr': plot_train_err})
    sio.savemat("results/result_test_err.mat", mdict={'testErr': plot_test_err})
    sio.savemat("results/result_beta.mat", mdict={'beta': plot_beta})
    sio.savemat("results/result_hsp.mat", mdict={'hsp': plot_hsp})

else:
    None 